# SIXT33N version A
## Phase 2: Principal Component Analysis

### EE 16B: Designing Information Devices and Systems II, Fall 2015

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-

##Table of Contents

* [Introduction](#intro)
* [Part 1: Data Collection](#part1)
* [Part 2: Principal Component Analysis](#part2)
* [Part 3: Classification](#part3)
* [Part 4: Launchpad Implementation](#part4)

<a id='intro'></a>
## Introduction

In this phase you will experiment on different musical genres, and develop a reasonably accurate PCA classifier. You will acquire data using the Launchpad and microphone circuit, then stream it to the PC. You develop a model in this iPython Notebook (since iPython has pretty graphs and lots of computing power, and a Launchpad does not). You will then experiment with different methods of signal processing to develop a classifier algorithm. You will also examine ways to optimize the classification and evaluate the accuracy of the prediction. 

Once you (and your GSI) are satisfied with the classifier's accuracy, you will port the classifier into the Launchpad code in Energia. <b>You will need to write a report explaing your design choices, methodology, and the classifying algorithm you created.</b> As a reminder, you will have 2 weeks to work on this phase since version A is heavy on PCA.

The goals of this phase are as follows:
- Experiment with different genres
- PCA + Classifier (2 genres)
- Check accuracy
- PCA projection on Launchpad

<a id='part1'></a>
##<span style="color:blue">Part 1: Data Collection</span>

###Materials
- Microphone front-end circuit
- Launchpad + USB

Brainstorm, with your partner, several music genres that you think will classify well (you will definitely want more than two). You will be collecting data in the frequency domain, so be sure to choose genres that you think will have large differences in that domain.

**<span style="color:red">What genres are you going to try? Why?</span>** 

Make sure that your microphone from the last stage is still working - remember to power the first op-amp from the voltage regulator and the second op-amp from the Launchpad 3.3V pin. Probe the output voltage of the front end circuit and make sure the DC level is around 1.6V and the signal saturates at 0V and 3.3V. 

Upload the sketch <b>`collect-data-music.ino`</b> from the last part of the previous phase. The sketch gathers ADC samples every 0.35ms, performs a 128-point FFT, and streams the real and imaginary parts of the coefficient back to the PC. After the code is uploaded, hit the reset button. Choose one of the generes of music that you would like to test and play it on your lab speakers. Once you confirm that your microphone is close enough to the speakers to hear the mucis, run the script:

<b>`python collect-data.py log.csv`</b>

This writes the real and imaginary parts of the FFT coefficients in `log.csv`. For each genre, you need at least around 600 data points (600 rows in your .csv), so leave a playlist running for a while. You can always go back to this step later if the genres are too hard to classify. Once the data is collected, rename `log.csv` to something descriptive of the genre you just played. Repeat this process for your other genres.

<a id='part2'></a>
##<span style="color:blue">Part 2: Principal Component Analysis</span>

Here's where your creativity comes in. The code below should read the .csv files you recorded in the previous part. Call read_csv() so that each file is parsed into a different variable.

In [ ]:
import numpy as np
import numpy.matlib
import scipy.io
import matplotlib.pyplot as plt
import csv
%matplotlib inline

In [ ]:
def read_csv(filename):
    """
    Reads a csv file and returns the first 20 recordings from the file
    Input:
        filename: csv filename
    Output:
        data: a 20x66 matrix corresponding to the first 20 readings in the csv file. Each row corresponds
            to a reading; the first 33 values are x-coordinates while the second33 values are y-coordinates
    """
    data = []
    with open(filename, 'r') as csvfile:
        r = csv.reader(csvfile, delimiter=' ')
        for row in r:
            data.append([float(i) for i in row[0].split(',')])
    return np.array(data)

In [ ]:
# Read the csv files using read_csv
# YOUR CODE HERE
genre1 = read_csv(...)
genre2 = read_csv(...)
...

# Take the same number of readings for both genres to be fair
num_samples = min(np.shape(genre1)[0], np.shape(genre2)[0]), ...
genre1 = genre1[:num_samples,:]
genre2 = genre2[:num_samples,:]
...

In [ ]:
# Plot the readings
plt.plot(genre1.T)
plt.figure()
plt.plot(genre2.T)
plt.show()
...

Look at the plots you generated. Which two genres do you think PCA will be able to tell apart the best? If you are having trouble telling apart the genres, PCA will probably have trouble doing so as well.

For the following code, you will use the two genres that you think PCA will classify the best.

You will now write the function <b>`build_A`</b> below that builds the A matrix for the Principal Component Analysis. You probably want to process the data in some way to a more suitable form for PCA. Experiment with a couple methods of signal processing.

- How much of your FFT is useful. (Hint: Your signal is real. Hint Hint: what is the difference between fft and rfft?)
- Try experimenting with using different ranges of your spectrum. 
- You have data for both real and imaginary parts of your FFT, does one work better than the other? You might also want to think about ways to combine the real and imaginary parts of each coefficient and somehow normalize each row.

Keep in mind that the same data pre-processing algorithm has to run on the Launchpad, so try to keep it as simple as possible.

In [ ]:
def build_A(data):
    """
    Build the A matrix for PCA. This may include some data manipulation.
    Input:
        data: a Nx256 matrix containing 1 FFT data reading in each row. The real and imaginary
            parts of each coefficient are interleaved, so the first row contains
            [re0 im0 re1 im1 ... re127 im127] if reN is the real part of the Nth coefficient
            and imN is the imaginary part of the Nth coefficient
    Output:
        A: the A matrix for PCA
    """
    sA = np.shape(data)
    
    # YOUR CODE HERE
    A = 
    for i in range(len(data)):
        # YOUR CODE HERE
        # plt.plot(...) You can plot each row to see how your data processing changes the data
    return A

In [ ]:
# Build A matrix for PCA
A = build_A(genre1)
A = np.vstack((A, build_A(genre2)))

sA = np.shape(A)

In [ ]:
# Show the mean of each 50 samples in each genre after the data processing in the build_A function
for i in range(num_samples//50):
    plt.plot(np.mean(A[i*50:(i+1)*50], 0), 'red')
for i in range(num_samples//50):
    plt.plot(np.mean(A[num_samples + i*50 : num_samples + (i+1)*50], 0), 'blue')

Try writing the following SVD code on your matrix A, and get a base line of how well it works.

Once you get an idea of what it looks like on its own, you might want to experiment with using different pre-processing on your matrix A. For example, zero-meaning might have interesting results. Let A2 be A with some signal processing applied to it.


**<span style="color:red">What processing techniques did you try on the matrix and and what was the result?</span>** 


In [ ]:
A2 = A

Try using SVD to retrieve the principal components. After you have done so, plot the sigma values. If they are not satisfactory, think of other ways you can change `build_A` to get a better result. The result might not be as nice as you expected, which is fine - we will try to cover this fact in the classification.

In [ ]:
# Take the SVD of A2
# YOUR CODE HERE
[u,s,v] = 

In [ ]:
# Plot the sigma values
plt.stem(s)
plt.xlim([-0.5,10])
plt.title('Sigma values')

Now plot out the significant principal component(s) you found above, project the data on the new space and plot the projection.

In [ ]:
# Plot the significant principal component
# YOUR CODE HERE


In [ ]:
# Project the data matrix to the first principal component
# YOUR CODE HERE
proj = 

# Plot the projection
x = np.linspace(0,num_samples, num_samples)
plt.scatter(x,proj[:num_samples], c=['red'], edgecolors='none', alpha=0.7)
plt.scatter(x + num_samples, proj[num_samples:], c=['blue'], edgecolors='none', alpha=0.7)
plt.legend(['genre1', 'genre2'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

You can also try plotting a histogram of the projection results to see how each genre behaves with the PCA vectors.

In [ ]:
plt.hist(proj[:num_samples], 100, color=['red'], alpha=0.5, histtype='stepfilled')
plt.hist(proj[num_samples:], 100, color=['blue'], alpha=0.5, histtype='stepfilled')
plt.legend(['genre1', 'genre2'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

Your data might look noisy, but this is very noisy data that we are classifying so some noise is okay. The important part is if you think that you can see some clustering. Once you think you have decent clustering, you can move on to getting your code to classify your data for you. However, if you don't see clustering, go back and and see what you can do to improve it. If you cannot separate your data with some decent accuracy, your code probably cannot either.

<a id='part3'></a>
##<span style="color:blue">Part 3: Classification</span>

Using the plots above, define a way of classifying the different gestures. Fill in the skeleton code below to determine the gesture of a new reading vector and try out the classification function. Don't forget to do the same normalization to the vector if needed as we are feeding in raw data after <b>`build_A`</b>.

In [ ]:
def classify(vector, new_basis):
    """
    Classifies a new reading vector into one of the 2 genres.
    Inputs:
        vector: 1xM reading vector - pre-processed data vector (i.e. a line of A - NOT A2)
        new_basis: matrix with the basis of the new space
    Output:
        String of the classified genre
    """
    # YOUR CODE HERE
    proj = 
    if (...):
        return 'genre1'
    if (...):
        return 'genre2'

In [ ]:
# Try out the classification function
print(classify(A[0,:], ...)) # Modify to use other vectors

Let's see the accuracy of your classification. <b>Our target is for the classifier to classify at least 80% correctly on both genres.</b> Apply the `classify` function to each row in <b>`A`</b> and check whether the classification is accurate. <b>Compute the accuracy</b> of classifying genre1 correctly and the accuracy of classifying genre2 correctly. These values should be about equal, and large differences in them might help you identify what needs to change in your <b>`classify`</b> function. Don't worry if the accuracy is not high enough - we will fix that soon.

In [ ]:
# Try to classify the whole A matrix
genre1_good = 0;
genre1_bad = 0;
genre2_good = 0;
genre2_bad = 0;

for row in A:
    # YOUR CODE HERE
    

Are the accuracy numbers high enough? If the accuracy still is not as high, think of other ways to make the classifier better. Consider collecting multiple PCA projection results before deciding which genre the music belongs to, using some sort of majority function to choose the final answer.

In [ ]:
# Experiment with different methods to get the accuracy of classification 80% on both genres
# YOUR CODE HERE
genre1_good = 0;
genre1_bad = 0;
genre2_good = 0;
genre2_bad = 0;

for (i,row) in enumerate(A):
    # YOUR CODE HERE

<a id='part4'></a>
##<span style="color:blue">Part 4: Launchpad Implementation</span>

###Materials
- Microphone front-end circuit
- Launchpad + USB

Lastly, implement your <b>data processing</b> and <b>classification</b> (not the PCA) in a copy of the Launchpad sketch <b>`classify.ino`</b>. Modify the sketch such that the green LED turns on if it identifies the first genre and turns off if it identifies the second genre. You can delete the code that prints out the FFT coefficients. To turn on the green LED, use the following code:

<b>`digitalWrite(GREEN_LED, HIGH);`</b>

and to turn it off, use the following code:

<b>`digitalWrite(GREEN_LED, LOW);`</b>

If you have some trouble compiling the sketch (for example the RAM is not big enough), you might need to optimize your algorithms or reuse variables and arrays. Seek help from a GSI if this happens.

Verify that the classification works most of the time by playing different songs. Once you are happy with the genre identification, you're done! <b>Do not forget to document all the methods and algorithms you used in this document for the final project report.</b>